In [1]:
import requests
import pyconll
from sklearn import metrics
from tqdm import tqdm

In [2]:
def askLamma(prompt):
    api_url = "http://127.0.0.1:8042/llama/"
    todo = {"prompts":[prompt], "max_gen_len": 500}
    response = requests.post(api_url, json=todo)
    resp = response.json()
    predicted_text = resp['responses'][0]['generation']
    endOfAnswer = predicted_text.find('\n')
    return predicted_text[:endOfAnswer]

In [3]:
# reading first 1020 dataset sentences
frases = pyconll.load_from_file('data/porttinari-base/Porttinari-base_test.conllu')
reviews = []
for sent in frases[:1020]:
    lista_ud=[]
    for token in sent:
        lista_ud.append([token.form, token.xpos, token.upos, token.deprel])
    reviews.append(lista_ud)

In [4]:
promptStart = "Atuando como linguista, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies) como nos exemplos abaixo.\n\n"
for review in reviews[:10]:
    entrada = "Entrada: "
    saida  = "Saida: "
    for token in review:
        if not token[2]:
            token[2] = 'None'
        entrada += token[0] + ' '
        saida += token[0] + '/' + token[2] + ' '
    entrada = entrada.strip()
    saida = saida.strip()
    promptStart += entrada + "\n" + saida + "\n\n"
#print(promptStart)

In [5]:
len(promptStart)

3034

In [ ]:
gold = []
pred = []
totalTries = []
errors = 0
for review in tqdm(reviews[20:]):
    prompt = promptStart
    entrada = "Entrada: "
    saida  = "Saida: "
    goldSent = []
    for token in review:
        if not token[2]:
            token[2] = 'None'
        gold.append(token[2])
        goldSent.append(token[2])
        entrada += token[0] + ' '
        saida += token[0] + '/' + token[2] + ' '
    entrada = entrada.strip()
    saida = saida.strip()
    prompt += entrada + "\nSaída: "
    tries = 0
    while True:
        answer = askLamma(prompt)
        tries += 1
        #print(answer)
        try:
            tagsSent = [token.split('/')[1] for token in answer.split(' ')]
        except:
            #print('retry except', tries)
            continue
        if len(tagsSent) == len(goldSent):
            #print('retry', tries)
            totalTries.append(tries)
            break
        if tries > 15:
            errors += 1
            print('error')
            tagsSent = ['None'] * len(goldSent)
            totalTries.append(tries)
            break
    pred += tagsSent
    #print('=======')
print('gold',len(gold),'pred',len(pred),'errors',errors)
print(metrics.classification_report(gold, pred))

  0%|▎                                                                                                                                                                        | 2/1000 [00:21<2:57:54, 10.70s/it]

In [7]:
totalTries

[1, 1, 2, 1]